In [1]:
import pandas as pd
import os
import janitor
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns; sns.set_style("darkgrid")
%matplotlib inline
import matplotlib.pyplot as plt

## Does the team with the highest FPL points win the premier league?

In [2]:
new_df = pd.read_excel("../data/fpl_data/full_df.xlsx")

In [3]:
new_df

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,season
0,Aaron Connolly,FWD,Brighton,0.5,0,0,-3,0,0.3,78,...,55,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
1,Aaron Cresswell,DEF,West Ham,2.1,0,0,11,0,11.2,435,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
2,Aaron Mooy,MID,Brighton,0.0,0,0,0,0,0.0,60,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
3,Aaron Ramsdale,GK,Sheffield Utd,2.5,0,0,12,0,0.0,483,...,50,True,0,1,0.00,0.00,0.00,0.00,0,2020/21
4,Abdoulaye Doucouré,MID,Everton,1.3,0,0,20,1,44.6,512,...,55,False,0,1,0.00,0.00,0.00,0.00,0,2020/21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93194,Kyle Walker,DEF,Man City,3.7,0,0,16,0,36.5,369,...,55,False,0,23,0.12,0.14,0.02,1.02,1,2023/24
93195,Jacob Brown,FWD,Luton,-0.7,0,0,1,0,0.5,631,...,49,False,0,23,0.00,0.05,0.05,0.19,0,2023/24
93196,Vicente Guaita,GK,Crystal Palace,0.0,0,0,0,0,0.0,227,...,44,False,0,23,0.00,0.00,0.00,0.00,0,2023/24
93197,Braian Ojeda Rodríguez,MID,Nott'm Forest,0.0,0,0,0,0,0.0,459,...,45,False,0,23,0.00,0.00,0.00,0.00,0,2023/24


In [4]:
df_2020_21 = new_df[new_df["season"] == "2020/21"]
df_2021_22 = new_df[new_df["season"] == "2021/22"]
df_2022_23 = new_df[new_df["season"] == "2022/23"]
df_2023_24 = new_df[new_df["season"] == "2023/24"]

In [5]:
df_2020_21.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24365 entries, 0 to 24364
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        24365 non-null  object 
 1   position                    24365 non-null  object 
 2   team                        24365 non-null  object 
 3   xP                          24365 non-null  float64
 4   assists                     24365 non-null  int64  
 5   bonus                       24365 non-null  int64  
 6   bps                         24365 non-null  int64  
 7   clean_sheets                24365 non-null  int64  
 8   creativity                  24365 non-null  float64
 9   element                     24365 non-null  int64  
 10  fixture                     24365 non-null  int64  
 11  goals_conceded              24365 non-null  int64  
 12  goals_scored                24365 non-null  int64  
 13  ict_index                   24365 no

In [6]:
import pandas as pd
import numpy as np

def accumulated(df):
    # Add a column 'n' with all values set to 1
    df['n'] = 1
    
    # Group by 'name' and 'team'
    grouped = df.groupby(['name', 'team'])
    
    # Define the columns to apply the cumulative sum and then subtract the original value
    cols_to_accumulate = ['goals_scored', 'assists', 'ict_index', 'goals_conceded', 'minutes', 'own_goals', 'bps', 'clean_sheets', 'bonus']
    
    # Apply the cumulative sum and subtraction
    for col in cols_to_accumulate:
        df[col + '_cum'] = grouped[col].cumsum() - df[col]
    
    # For 'total_points', just the cumulative sum is needed
    df['total_points_cum'] = grouped['total_points'].cumsum()
    
    # Select the desired columns
    df = df[['name', 'team', 'position', 'goals_scored_cum', 'assists_cum', 'ict_index_cum', 'goals_conceded_cum', 'minutes_cum', 'own_goals_cum', 'total_points_cum', 'bps_cum', 'bonus_cum', 'clean_sheets_cum', 'GW', 'kickoff_time']]
    
    # Filter rows where 'ict_index_cum' > 0
    df = df[df['ict_index_cum'] > 0]
    
    # Drop the temporary 'n' column
    #df.drop(columns=['n'], inplace=True)
    
    # Rename the accumulated columns back to their original names for clarity
    df.rename(columns={col + '_cum': col for col in cols_to_accumulate}, inplace=True)
    df.rename(columns={'total_points_cum': 'total_points'}, inplace=True)
    
    return df

In [7]:
df_2020_21c = accumulated(df_2020_21).reset_index(drop=True)

In [8]:
df_2020_21c[df_2020_21c["GW"]==38].groupby("team", as_index=False)["total_points"].sum()

,team,total_points
0,Arsenal,1644
1,Aston Villa,1703
2,Brighton,1459
3,Burnley,1375
4,Chelsea,1880
5,Crystal Palace,1378
6,Everton,1584
7,Fulham,1282
8,Leeds,1691
9,Leicester,1679


In [9]:
def fpl_rankings(df):
    dfc = accumulated(df).reset_index(drop=True)
    dfc = dfc[dfc["GW"]==38].groupby("team", as_index=False)["total_points"].sum().sort_values("total_points", ascending=False).reset_index(drop=True)
    return dfc

In [10]:
fpl_rankings(df_2020_21)

,team,total_points
0,Man City,2124
1,Chelsea,1880
2,Man Utd,1812
3,Liverpool,1761
4,Spurs,1759
5,Aston Villa,1703
6,Leeds,1691
7,West Ham,1686
8,Leicester,1679
9,Arsenal,1644


In [11]:
fpl_rankings(df_2021_22)

,team,total_points
0,Liverpool,2306
1,Man City,2260
2,Chelsea,2017
3,Spurs,1874
4,Arsenal,1690
5,West Ham,1610
6,Crystal Palace,1609
7,Leicester,1567
8,Wolves,1535
9,Man Utd,1515


In [12]:
fpl_rankings(df_2022_23)

,team,total_points
0,Man City,2050
1,Arsenal,2023
2,Newcastle,1877
3,Liverpool,1852
4,Man Utd,1836
5,Brentford,1730
6,Brighton,1686
7,Spurs,1680
8,Aston Villa,1574
9,Fulham,1550


In [13]:
df_2023_24[df_2023_24["name"] == "Mohamed Salah"]

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,value,was_home,yellow_cards,GW,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,season
76868,Mohamed Salah,MID,Liverpool,4.5,1,0,15,0,20.2,308,...,125,False,0,1,0.31,0.59,0.28,1.28,1,2023/24
77475,Mohamed Salah,MID,Liverpool,5.5,0,0,15,0,33.0,308,...,125,True,0,2,0.11,1.34,1.23,1.00,1,2023/24
78142,Mohamed Salah,MID,Liverpool,5.0,1,0,19,0,24.9,308,...,125,False,0,3,0.12,0.30,0.18,2.00,1,2023/24
78842,Mohamed Salah,MID,Liverpool,6.3,0,0,26,1,23.1,308,...,125,True,0,4,0.07,0.87,0.80,0.66,1,2023/24
79556,Mohamed Salah,MID,Liverpool,8.2,2,2,35,0,57.4,308,...,125,False,0,5,1.47,1.71,0.24,0.63,1,2023/24
80274,Mohamed Salah,MID,Liverpool,8.7,0,3,36,0,28.8,308,...,125,True,0,6,0.27,1.28,1.01,1.11,1,2023/24
81063,Mohamed Salah,MID,Liverpool,7.5,0,0,3,0,27.8,308,...,125,False,1,7,0.39,0.44,0.05,1.44,1,2023/24
81803,Mohamed Salah,MID,Liverpool,9.2,0,3,43,0,13.7,308,...,126,False,0,8,0.07,1.52,1.45,2.30,1,2023/24
82532,Mohamed Salah,MID,Liverpool,11.2,0,3,40,1,34.1,308,...,127,True,0,9,0.05,1.44,1.39,0.13,1,2023/24
83263,Mohamed Salah,MID,Liverpool,13.5,0,0,19,1,32.3,308,...,128,True,0,10,0.13,0.71,0.58,0.45,1,2023/24


In [14]:
df_2023_24.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16882 entries, 76317 to 93198
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   name                        16882 non-null  object 
 1   position                    16882 non-null  object 
 2   team                        16882 non-null  object 
 3   xP                          16882 non-null  float64
 4   assists                     16882 non-null  int64  
 5   bonus                       16882 non-null  int64  
 6   bps                         16882 non-null  int64  
 7   clean_sheets                16882 non-null  int64  
 8   creativity                  16882 non-null  float64
 9   element                     16882 non-null  int64  
 10  fixture                     16882 non-null  int64  
 11  goals_conceded              16882 non-null  int64  
 12  goals_scored                16882 non-null  int64  
 13  ict_index                   1688

### General EDA